Cross Validation of SARIMAX Forecasting Nuitees dans l'hôtellerie Pays de La Loire PDL 2016 
Set "stepwise = true" in auto-arima. 
Include 3 exogenous variables (meteo,GDP,holidays) in training & test datasets
Predict GDP with auto-arima for 2016 & 2017 and include as exogenous in training&test datasets
Datasets https://github.com/jinnyto/touristcast/tree/datasets/datasets

In [1]:
import pandas as pd
import numpy as np
import datetime 
import matplotlib.pyplot as plt
import pyramid
%matplotlib inline
from statsmodels.tsa.seasonal import seasonal_decompose
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from math import sqrt
def seasonal_MASE(truth, forecast, seasonality=1):    
    period = truth.shape[0] # T
    # print(period)
    forecast_errors = np.abs(truth - forecast)
    # print(forecast_errors)
    mean_absolute_forecast_error = np.sum(forecast_errors) / period
    # print(mean_absolute_forecast_error)

    naive_period = truth.shape[0] - seasonality # T - m
    # print(naive_period)
    # print(truth[seasonality:])
    # print(truth[:period - seasonality])
    naive_errors = np.abs(truth[seasonality:] - truth[:period - seasonality])
    mean_absolute_naive_error = np.sum(naive_errors) / naive_period
    
    return mean_absolute_forecast_error / mean_absolute_naive_error

In [2]:
dn = pd.read_csv('./datasets/nights_2010-2017.csv',parse_dates=['date'], index_col='date',usecols=[*range(1, 15)])
dm = pd.read_csv('./datasets/avgtemp_2010-2017.csv',parse_dates=['date'],index_col='date',usecols=[*range(1, 15)])
dh = pd.read_csv('./datasets/daysoff_2010-2017.csv',parse_dates=['date'],index_col='date',usecols=['date','daysoff'])
dg = pd.read_csv('./datasets/regionalGDP_2010-2015.csv',parse_dates=['date'],index_col='date',usecols=[*range(1, 15)])
dr = pd.concat([dn, dm,dh,dg], axis=1)
df = dr.dropna()
df = df[['nights_PDL','avgtemp_PDL','gdp_PDL','daysoff']]
exogenous= np.array(df[['avgtemp_PDL','gdp_PDL','daysoff']])
y = np.array(df.nights_PDL)

In [3]:
from pyramid.arima import auto_arima
tc_fit = auto_arima(y=y,exogenous=exogenous,start_p=1, start_q=1, max_p=3, max_q=3, m=12,
                    start_P=1, seasonal=True, d=1, D=1, trace=True,
                    error_action='ignore',  # don't want to know if an order does not work
                    suppress_warnings=True,  # don't want convergence warnings
                    stepwise=True)
tc_fit.summary()

Fit ARIMA: order=(1, 1, 1) seasonal_order=(1, 1, 1, 12); AIC=560.165, BIC=578.863, Fit time=2.256 seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 1, 0, 12); AIC=598.781, BIC=609.169, Fit time=0.095 seconds
Fit ARIMA: order=(1, 1, 0) seasonal_order=(1, 1, 0, 12); AIC=573.443, BIC=587.986, Fit time=1.150 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 1, 1, 12); AIC=552.487, BIC=567.030, Fit time=1.660 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(1, 1, 1, 12); AIC=554.987, BIC=571.608, Fit time=2.037 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 1, 0, 12); AIC=560.782, BIC=573.247, Fit time=0.751 seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(0, 1, 2, 12); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(0, 1, 1) seasonal_order=(1, 1, 2, 12); AIC=nan, BIC=nan, Fit time=nan seconds
Fit ARIMA: order=(1, 1, 1) seasonal_order=(0, 1, 1, 12); AIC=555.042, BIC=571.662, Fit time=2.006 seconds
Fit ARIMA: order=(0, 1, 0) seasonal_order=(0, 1, 1, 12); AIC=581.2

<class 'statsmodels.iolib.summary.Summary'>
"""
                                 Statespace Model Results                                 
==========================================================================================
Dep. Variable:                                  y   No. Observations:                   72
Model:             SARIMAX(0, 1, 1)x(0, 1, 1, 12)   Log Likelihood                -269.243
Date:                            Wed, 29 Aug 2018   AIC                            552.487
Time:                                    22:38:40   BIC                            567.030
Sample:                                         0   HQIC                           558.164
                                             - 72                                         
Covariance Type:                              opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.3086      0.217      1.423      0.155      -0.116       0.734
x1            -0.7103      1.713     -0.415      0.678      -4.068       2.647
x2             0.0121      0.003      4.452      0.000       0.007       0.017
x3             5.7685      2.848      2.026      0.043       0.187      11.350
ma.L1         -0.9908      0.565     -1.753      0.080      -2.099       0.117
ma.S.L12      -0.5001      0.211     -2.373      0.018      -0.913      -0.087
sigma2       469.8726    244.215      1.924      0.054      -8.780     948.525
===================================================================================
Ljung-Box (Q):                       28.26   Jarque-Bera (JB):                 2.40
Prob(Q):                              0.92   Prob(JB):                         0.30
Heteroskedasticity (H):               1.08   Skew:                             0.43
Prob(H) (two-sided):                  0.87   Kurtosis:                         2.53
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [4]:
mc_fit = auto_arima(y=np.array(df.gdp_PDL),start_p=0, start_q=0, max_p=3, max_q=3, m=12,
                    start_P=1, seasonal=False, d=1, D=1, trace=True,
                    error_action='ignore',  # don't want to know if an order does not work
                    suppress_warnings=True,  # don't want convergence warnings
                    stepwise=True)
mc_fit.summary()

Fit ARIMA: order=(0, 1, 0); AIC=1148.679, BIC=1153.204, Fit time=0.039 seconds
Fit ARIMA: order=(1, 1, 0); AIC=1150.440, BIC=1157.228, Fit time=0.126 seconds
Fit ARIMA: order=(0, 1, 1); AIC=1150.404, BIC=1157.192, Fit time=0.022 seconds
Fit ARIMA: order=(1, 1, 1); AIC=nan, BIC=nan, Fit time=nan seconds
Total fit time: 0.208 seconds


<class 'statsmodels.iolib.summary.Summary'>
"""
                             ARIMA Model Results                              
==============================================================================
Dep. Variable:                    D.y   No. Observations:                   71
Model:                 ARIMA(0, 1, 0)   Log Likelihood                -572.340
Method:                           css   S.D. of innovations            766.766
Date:                Wed, 29 Aug 2018   AIC                           1148.679
Time:                        22:38:44   BIC                           1153.204
Sample:                             1   HQIC                          1150.479
                                                                              
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const        183.9145     90.998      2.021      0.047       5.561     362.268
==============================================================================
"""

In [5]:
mc_fit.fit(df.gdp_PDL)

ARIMA(callback=None, disp=0, maxiter=50, method=None, order=(0, 1, 0),
   out_of_sample_size=0, scoring='mse', scoring_args={},
   seasonal_order=None, solver='lbfgs', start_params=None,
   suppress_warnings=True, transparams=True, trend='c')

In [16]:
next_24 = mc_fit.predict(n_periods=24)
dr.loc['2016-01-01':'2017-12-01','gdp_PDL'] = next_24

In [17]:
smase_fold= []
startyear = '2010-01-01'
for index in range(2011,2018):
    nextyear = str(index)+'-01-01'
    predictyear = str(index+1)+'-01-01'
    dftrain = dr[(dr.index>=startyear)&(dr.index<nextyear)]
    train_nights  = np.array(dftrain.nights_PDL)
    train_exogenous =  np.array(dftrain[['avgtemp_PDL','gdp_PDL','daysoff']])
    dftest = dr[(dr.index>=nextyear)&(dr.index<predictyear)]
    test_nights = np.array(dftest.nights_PDL)
    test_exogenous =  np.array(dftest[['avgtemp_PDL','gdp_PDL','daysoff']])
    tc_future_forecast = tc_fit.predict(n_periods=12,exogenous = test_exogenous)
    smase_fold.append(seasonal_MASE(test_nights,tc_future_forecast))

print(smase_fold)
np.sum(smase_fold) / len(smase_fold)

[1.2844776456958755, 0.9504976564429287, 0.5253877670577137, 0.23390112049851, 0.15301361037334796, 0.17970474645596748, 0.2261106388874321]


0.5075847407731108